# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [85]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [86]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [87]:
# initiating an empty list of rows that will be generated from each file

full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [88]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.  Apache Cassandra Modeling

#### Creating a Cluster

In [89]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [90]:
# Create a Keyspace
keyspace_statement = """
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
"""

try:
    session.execute(keyspace_statement)
except Exception as e:
    print(e)

#### Set Keyspace

In [91]:
# Set KEYSPACE

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [93]:
# Create table

create_sessions_table_statement = """
CREATE TABLE IF NOT EXISTS sessions
(session_id INT, item INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY(session_id, item))
"""

try:
    session.execute(create_sessions_table_statement)
except Exception as e:
    print(e)

In [94]:
# Insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query  = "INSERT INTO sessions (session_id, item, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [95]:
# Verify Data Has Been Inserted

query = "SELECT * FROM sessions WHERE session_id = 338 AND item = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print(list(map(lambda row: row, rows)))

[Row(session_id=338, item=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')]


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [96]:
# Create table

create_user_sessions_table_statement = """
CREATE TABLE IF NOT EXISTS user_sessions
(user_id INT, session_id INT, item INT, artist TEXT, song TEXT, user TEXT,
PRIMARY KEY(user_id, session_id, item))
"""

try:
    session.execute(create_user_sessions_table_statement)
except Exception as e:
    print(e)

In [97]:
# Insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query  = "INSERT INTO user_sessions (user_id, session_id, item, artist, song, user)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"
        name = f"{line[1]} {line[4]}"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], name))

In [98]:
# Verify Data Has Been Inserted

query = """
SELECT *
FROM user_sessions
WHERE user_id = 10 AND session_id = 182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print(list(map(lambda row: row, rows)))

[Row(user_id=10, session_id=182, item=0, artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz'), Row(user_id=10, session_id=182, item=1, artist='Three Drives', song='Greece 2000', user='Sylvie Cruz'), Row(user_id=10, session_id=182, item=2, artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz'), Row(user_id=10, session_id=182, item=3, artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')]


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [99]:
# Create table

create_song_plays_table_statement = """
CREATE TABLE IF NOT EXISTS song_plays
(song TEXT, user TEXT, PRIMARY KEY(song, user))
"""

try:
    session.execute(create_song_plays_table_statement)
except Exception as e:
    print(e)

In [100]:
# Insert data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query  = "INSERT INTO song_plays (song, user)"
        query = query + "VALUES(%s, %s)"
        name = f"{line[1]} {line[4]}"
        session.execute(query, (line[9], name))

In [101]:
# Verify Data Has Been Inserted

query = "SELECT * FROM song_plays WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print(list(map(lambda row: row, rows)))

[Row(song='All Hands Against His Own', user='Jacqueline Lynch'), Row(song='All Hands Against His Own', user='Sara Johnson'), Row(song='All Hands Against His Own', user='Tegan Levine')]


## Drop the tables before closing out the sessions

In [102]:
session.execute("DROP TABLE IF EXISTS sessions")
session.execute("DROP TABLE IF EXISTS user_sessions")
session.execute("DROP TABLE IF EXISTS song_plays")

## Close the session and cluster connection¶

In [104]:
session.shutdown()
cluster.shutdown()